In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [ ]:
tf.__version__

'2.15.0'

## **Data Preprocessing**

In [ ]:
dataset = pd.read_csv('/content/Churn_Modelling.csv')
X = dataset.iloc[:, 3:-1]
y = dataset.iloc[:,-1]

In [ ]:
X.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,Female,42,2,0.00,1,1,1,101348.88
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58
2,502,France,Female,42,8,159660.80,3,1,0,113931.57
3,699,France,Female,39,1,0.00,2,0,0,93826.63
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10


In [ ]:
y.head()

0    1
1    0
2    1
3    0
4    0
Name: Exited, dtype: int64

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X.iloc[:,2] = le.fit_transform(X.iloc[:,2])

<ipython-input-8-a0451b3043b1>:3: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  X.iloc[:,2] = le.fit_transform(X.iloc[:,2])


In [ ]:
X['Gender'].unique()

array([0, 1])

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder',OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [ ]:
print(X)

[[1.0000000e+00 0.0000000e+00 0.0000000e+00 ... 1.0000000e+00
  1.0000000e+00 1.0134888e+05]
 [0.0000000e+00 0.0000000e+00 1.0000000e+00 ... 0.0000000e+00
  1.0000000e+00 1.1254258e+05]
 [1.0000000e+00 0.0000000e+00 0.0000000e+00 ... 1.0000000e+00
  0.0000000e+00 1.1393157e+05]
 ...
 [1.0000000e+00 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
  1.0000000e+00 4.2085580e+04]
 [0.0000000e+00 1.0000000e+00 0.0000000e+00 ... 1.0000000e+00
  0.0000000e+00 9.2888520e+04]
 [1.0000000e+00 0.0000000e+00 0.0000000e+00 ... 1.0000000e+00
  0.0000000e+00 3.8190780e+04]]


In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=0)

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## **Building ANN**

### Initializing the ANN

In [ ]:
ann = tf.keras.models.Sequential()

### First hidden layer

In [ ]:
ann.add(tf.keras.layers.Dense(units=6,activation='relu'))

## Second hidden layer

In [ ]:
ann.add(tf.keras.layers.Dense(units=6,activation='relu'))

### Output layer

In [ ]:
ann.add(tf.keras.layers.Dense(units=1,activation='sigmoid'))

### Compiling ANN

In [ ]:
ann.compile(optimizer = 'adam', loss='binary_crossentropy', metrics = ['accuracy'])

### Training the ANN

In [ ]:
ann.fit(X_train,y_train, batch_size=32, epochs=100)

Epoch 1/100
250/250 [==============================] - 1s 2ms/step - loss: 0.5436 - accuracy: 0.7534
Epoch 2/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4530 - accuracy: 0.8065
Epoch 3/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4277 - accuracy: 0.8192
Epoch 4/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4172 - accuracy: 0.8260
Epoch 5/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4117 - accuracy: 0.8280
Epoch 6/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4082 - accuracy: 0.8292
Epoch 7/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4056 - accuracy: 0.8304
Epoch 8/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4032 - accuracy: 0.8296
Epoch 9/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4007 - accuracy: 0.8315
Epoch 10/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3973 - accura

In [ ]:
print(ann.predict(sc.transform([[1,0,0,600,1,40,3,60000,2,1,1,50000]]))>0.5)

1/1 [==============================] - 0s 127ms/step
[[False]]


In [ ]:
import numpy as np

# Assuming y_test is a pandas Series object
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)

# Convert y_test to a numpy array before reshaping
y_test_array = y_test.to_numpy()

# Reshape both y_pred and y_test_array
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test_array.reshape(len(y_test_array),1)),1))


63/63 [==============================] - 0s 1ms/step
[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


In [ ]:
y_pred

array([[False],
       [False],
       [False],
       ...,
       [False],
       [False],
       [False]])

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test,y_pred)
print(cm)
accuracy_score(y_test,y_pred)


[[1515   80]
 [ 193  212]]


0.8635